In [1]:
import cv2
import numpy as np

using laplacian to estimate blurriness

In [2]:
cap = cv2.VideoCapture('../lecture.mp4')
font                   = cv2.FONT_HERSHEY_SIMPLEX
bottomLeftCornerOfText = (10,500)
fontScale              = 1
fontColor              = (255,255,255)
lineType               = 2

while 1:
    ret, frame = cap.read()
    if not ret:
        break
        
    laplacian_var = cv2.Laplacian(frame, cv2.CV_64F).var()
    
    cv2.putText(frame,str(laplacian_var),
        bottomLeftCornerOfText, 
        font, 
        fontScale,
        fontColor,
        lineType)
    cv2.imshow('vid',frame)
    k = cv2.waitKey(20) & 0xff
    if k == 27:
        break

cv2.destroyAllWindows()
cap.release()

calculate homography each second when frame not blurry

In [63]:

MAX_FEATURES = 500
GOOD_MATCH_PERCENT = 0.15
 
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

def alignImages(im1, im2):
    factor = 4.
    small1 = cv2.resize(im1, (0,0), fx=1/factor, fy=1/factor) 
    small2 = cv2.resize(im2, (0,0), fx=1/factor, fy=1/factor)
    
    rects1, weights1 = hog.detectMultiScale(small1, winStride=(4, 4),
        padding=(8, 8), scale=1.05)
    rects2, weights2 = hog.detectMultiScale(small2, winStride=(4, 4),
        padding=(8, 8), scale=1.05)
    rects1 = [[int(i*factor) for i in r] for r in rects1]
    rects2 = [[int(i*factor) for i in r] for r in rects2]
    mask1 = np.ones(im1.shape[:2],dtype='uint8') * 255
    mask2 = np.ones(im2.shape[:2],dtype='uint8') * 255
    for (x, y, w, h) in rects1:
		cv2.rectangle(mask1, (x-w, y), (x + w*2, y + h), 0, -1)
    for (x, y, w, h) in rects2:
		cv2.rectangle(mask2, (x-w, y), (x + w*2, y + h), 0, -1)
    # Convert images to grayscale
    im1Gray = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
    im2Gray = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)

    # Detect ORB features and compute descriptors.
    orb = cv2.ORB_create(MAX_FEATURES)
    keypoints1, descriptors1 = orb.detectAndCompute(im1Gray, mask1)
    keypoints2, descriptors2 = orb.detectAndCompute(im2Gray, mask2)

    # Match features.
    matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
    matches = matcher.match(descriptors1, descriptors2, None)

    # Sort matches by score
    matches.sort(key=lambda x: x.distance, reverse=False)

    # Remove not so good matches
    numGoodMatches = int(len(matches) * GOOD_MATCH_PERCENT)
    matches = matches[:numGoodMatches]

    # Draw top matches
    imMatches = cv2.drawMatches(im1, keypoints1, im2, keypoints2, matches, None)
    cv2.imwrite("matches.jpg", imMatches)

    # Extract location of good matches
    points1 = np.zeros((len(matches), 2), dtype=np.float32)
    points2 = np.zeros((len(matches), 2), dtype=np.float32)

    for i, match in enumerate(matches):
        x1,y1 = keypoints1[match.queryIdx].pt
        x2,y2 = keypoints2[match.trainIdx].pt
        if abs(y1 - y2) >= 100:
            continue
        points1[i, :] = keypoints1[match.queryIdx].pt
        points2[i, :] = keypoints2[match.trainIdx].pt

    # Find homography
    h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)

    # reprojection error calculation
    points1t = np.hstack((points1,np.ones((points1.shape[0], 1))))
    homo_vec = (np.dot(h, points1t.T)).T
    dehomo_vec = homo_vec[:,:2] / homo_vec[:,2:3]
    error = np.linalg.norm(dehomo_vec-points2, axis=1).mean()
    
    # Use homography
    height, width, channels = im2.shape
    im1Reg = cv2.warpPerspective(im1, h, (width, height))

    return im1Reg, h, error

In [64]:
cap = cv2.VideoCapture('../lecture.mp4')
_, old_frame = cap.read()

while 1:
    for i in range(10):
        if not cap.grab():
            break # TODO: only breaks from inner loop...
    ret, frame = cap.read()
    if not ret:
        break
        
    laplacian_var = cv2.Laplacian(frame, cv2.CV_64F).var()
    if laplacian_var < 450.:
        print "skipped"
        continue
    aligned, h, error = alignImages(old_frame, frame)
#     sing_val = np.linalg.svd(h)[1]
#     str(sing_val[0]) + " " + str(sing_val[0]/sing_val[2]))
    blended = cv2.addWeighted(frame,.5,aligned,.5,0)
    cv2.putText(blended,str(error),
        bottomLeftCornerOfText, 
        font, 
        fontScale,
        fontColor,
        lineType)
    cv2.imshow('aligned', blended)
    if error < 10:
        cv2.imshow('good', frame)
        old_frame = frame
    k = cv2.waitKey(10) & 0xff
    if k == 27:
        break

cv2.destroyAllWindows()
cap.release()

skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
